In [2]:
import jpype
from jpype import JClass

In [4]:
jpype.startJVM()

In [5]:
JString = JClass('java.lang.String')

In [7]:
test_java_str = JString('hello world')

[Java String documentation](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/lang/String.html#method.summary)

In [10]:
', '.join(dir(test_java_str))

'CASE_INSENSITIVE_ORDER, __add__, __class__, __contains__, __del__, __delattr__, __dict__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __getitem__, __gt__, __hash__, __init__, __init_subclass__, __javadoc__, __le__, __len__, __lt__, __module__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __str__, __subclasshook__, __weakref__, charAt, chars, codePointAt, codePointBefore, codePointCount, codePoints, compare, compareTo, compareToIgnoreCase, concat, contains, contentEquals, copyValueOf, endsWith, equals, equalsIgnoreCase, format, getBytes, getChars, getClass, hashCode, indexOf, intern, isBlank, isEmpty, join, lastIndexOf, length, lines, matches, notify, notifyAll, offsetByCodePoints, regionMatches, repeat, replace, replaceAll, replaceFirst, split, startsWith, strip, stripLeading, stripTrailing, subSequence, substring, toCharArray, toLowerCase, toString, toUpperCase, trim, valueOf, wait'

In [11]:
test_python_str = 'hello world'

[Python String documentation](https://docs.python.org/3.8/library/stdtypes.html#str)

In [12]:
', '.join(dir(test_python_str))

'__add__, __class__, __contains__, __delattr__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __getitem__, __getnewargs__, __gt__, __hash__, __init__, __init_subclass__, __iter__, __le__, __len__, __lt__, __mod__, __mul__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __rmod__, __rmul__, __setattr__, __sizeof__, __str__, __subclasshook__, capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, format_map, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, maketrans, partition, replace, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill'

In [18]:
print(test_java_str)

hello world


In [21]:
print(f'python: {test_python_str}\njava: {test_java_str}')

python: hello world
java: hello world


In [22]:
test_java_str.toUpperCase()

'HELLO WORLD'

Here is the [first working prototype](https://github.com/hx2A/py5generator/blob/636712b1c786eee67a6e1cbec9d59d2a25afb0e8/packages/py5/py5/__init__.py):

Was using [pyjnius](https://github.com/kivy/pyjnius) back then


```
import jnius_config
jnius_config.set_classpath('.', '/home/jim/Projects/git/processing/core/library/*')
from jnius import autoclass  # noqa

PythonPApplet = autoclass('processing.core.PythonPApplet')
PConstants = autoclass('processing.core.PConstants')

_papplet = PythonPApplet()

# *** PY5 GENERATED CONSTANTS ***
ADD = 2
ALPHA = 4

# ...lots of constants...


# *** PY5 GENERATED FUNCTIONS ***
def abs(*args):
    return _papplet.abs(*args)


def acos(*args):
    return _papplet.acos(*args)

# ...lots of functions...


frame_rate = 0
mouse_x = 0
mouse_y = 0


def _handle_settings(settings):
    _papplet.handleSettingsPt1()
    settings()
    _papplet.handleSettingsPt2()


def _update_vars():
    global frame_rate
    frame_rate = _papplet.frameRate
    global mouse_x
    mouse_x = _papplet.mouseX
    global mouse_y
    mouse_y = _papplet.mouseY


def _handle_draw(setup, draw):
    _update_vars()
    _papplet.handleDrawPt1()
    if _papplet.frameCount == 0:
        setup()
    _papplet.handleDrawPt2()
    draw()
    _papplet.handleDrawPt3()

    _papplet.render()


def run_sketch(settings, setup, draw, frameLimit=1000):
    _handle_settings(settings)
    PythonPApplet.setupSketch(['py5 sketch'], _papplet)

    while frameLimit > 0:
        _handle_draw(setup, draw)
        time.sleep(1 / 60)

        frameLimit -= 1
```

The Java PythonPApplet class was inherited from Processing's PApplet class and had some extra functions to facilitate sketch execution. Most importantly, [Processing's handleDraw method](https://github.com/processing/processing4/blob/e96622d633cb83972be1b85e94065426e407fd36/core/src/processing/core/PApplet.java#L2369) was divided up into new `handleDrawPt1`, `handleDrawPt2`, and `handleDrawPt3` methods. The code was segmented before and after `handleDraw`'s calls to the user's `setup()` and `draw()` methods. This enabled me to have Processing execute the same Java code with the user's Python `setup()` and `draw()` methods called at the appropriate time. Processing's `handleSettings` method and the user's `settings()` method are managed in a similar way.

This worked when using the default Processing renderer (JAVA2D), which uses [AWT](https://docs.oracle.com/en/java/javase/11/docs/api/java.desktop/java/awt/package-summary.html) to draw to the screen. Unfortunately, neither of the OpenGL renderers worked (P2D and P3D). It was a struggle to understand the errors and get it to work.

First problem: error about the Animation thread not running

Solution: some complicated thing with an actual running sketch and simultaneous threads in Python and Java that would use a semaphore to pause Java and execute the Python `setup()` and `draw()` methods at the appropriate times

Second problem: error about [OpenGL Context](https://www.khronos.org/opengl/wiki/Creating_an_OpenGL_Context_(WGL))

# Final Version

To solve all these problems and move forward with the library I used functionality in both jpype and pyjnius to allow calls from Java to Python.

    Proxies in Java are foreign elements that pretend to implement a Java interface.

https://jpype.readthedocs.io/en/latest/userguide.html#implementing-java-interfaces

https://github.com/hx2A/py5generator/blob/master/py5_jar/src/py5/core/Py5Applet.java

The `Py5Applet` class extends Processing's PApplet class and implements all the user functions like so:

```
  @Override
  public void draw() {
    if (success) {
      if (py5RegisteredEvents.contains("draw")) {
        success = py5Methods.run_method("draw");
      } else {
        super.draw();
      }
    }
  }
```

That `py5methods` object implements this interface:

https://github.com/hx2A/py5generator/blob/master/py5_jar/src/py5/core/Py5Methods.java

Java thinks it is a real Java object, it it is actually implemented with the Py5Methods class, implemented in Python here:

https://github.com/hx2A/py5generator/blob/master/py5_resources/py5_module/py5/methods.py

Sketch is still running the animation thread

Python calls are on the same thread, resolving the OpenGL Context problems.

Python calls Processing's `runSketch()` method (Java), which opens the window, starts the animation sketch, runs the `Py5Applet` sketch. As far as Processing is concerned, this is regular sketch. `Py5Applet` makes calls back to Python via the Py5Methods interface. The user implemented `setup()` and `draw()` methods make calls back to Processing to draw the frame. Python calls Java calls Python calls Java.